In [1]:
#dependencies for helper functions/classes
import pandas as pd
import pyarrow.parquet as pq
from typing import NamedTuple
import os.path as path
import os
import progressbar
import requests
import numpy as np


#keras for ML
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout, Input, Dense
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.utils import plot_model, normalize
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD, Adam, Nadam, Adadelta
from tensorflow.keras.activations import relu, elu, sigmoid

#sklearn for preprocessing the data and train-test split
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, classification_report
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, r2_score, mean_squared_error, mean_absolute_error

#for plots
import matplotlib
import matplotlib.pyplot as plt

#%matplotlib inline

In [2]:
class Labels(NamedTuple):
    '''
    One-hot labeled data
    '''
    tissue: np.ndarray
    sex: np.ndarray
    age: np.ndarray
    death: np.ndarray
        

class Genes:
    '''
    Class to load GTEX samples and gene expressions data
    '''
    def __init__(self, samples_path: str = '', expressions_path: str = '', problem_type: str = "classification"):
        self.__set_samples(samples_path)
        self.__set_labels(problem_type)
        if expressions_path != '':
            self.expressions = self.get_expressions(expressions_path)

    def __set_samples(self, sample_path: str) -> pd.DataFrame:
        self.samples: pd.DataFrame = pq.read_table(sample_path).to_pandas()
        self.samples["Death"].fillna(-1.0, inplace = True)
        self.samples: pd.DataFrame = self.samples.set_index("Name")
        self.samples["Sex"].replace([1, 2], ['male', 'female'], inplace=True)
        self.samples["Death"].replace([-1,0,1,2,3,4], ['alive/NA', 'ventilator case', '<10 min.', '<1 hr', '1-24 hr.', '>1 day'], inplace=True)
        self.samples = self.samples[~self.samples['Death'].isin(['>1 day'])]
        return self.samples

    def __set_labels(self, problem_type: str = "classification") -> Labels:
        self.labels_list = ["Tissue", "Sex", "Age", "Death"]
        self.labels: pd.DataFrame = self.samples[self.labels_list]
        self.drop_list = self.labels_list + ["Subtissue", "Avg_age"]
        
        if problem_type == "classification":
            dummies_df = pd.get_dummies(self.labels["Age"])
            print(dummies_df.columns.tolist())
            self.Y = dummies_df.values
        
        if problem_type == "regression":
            self.Y = self.samples["Avg_age"].values
        
        return self.Y

    def sex_output(self, model):
        return Dense(units=self.Y.sex.shape[1], activation='softmax', name='sex_output')(model)

    def tissue_output(self, model):
        return Dense(units=self.Y.tissue.shape[1], activation='softmax', name='tissue_output')(model)

    def death_output(self, model):
        return Dense(units=self.Y.death.shape[1], activation='softmax', name='death_output')(model)

    def age_output(self, model):
        '''
        Created an output layer for the keras mode
        :param model: keras model
        :return: keras Dense layer
        '''
        return Dense(units=self.Y.age.shape[1], activation='softmax', name='age_output')(model)


    def get_expressions(self, expressions_path: str)->pd.DataFrame:
        '''
        load gene expressions DataFrame
        :param expressions_path: path to file with expressions
        :return: pandas dataframe with expression
        
        '''
        selected_genes = list(pd.read_csv('../data/gtex/gain_selected_features_tissues_False.csv')['ids'].values)
        
        if expressions_path.endswith(".parquet"):
            return pq.read_table(expressions_path).to_pandas().set_index("Name") #[selected_genes]
        else:
            separator = "," if expressions_path.endswith(".csv") else "\t"
            return pd.read_csv(expressions_path, sep=separator).set_index("Name") #[selected_genes]

    def prepare_data(self, normalize_expressions: bool = True)-> np.ndarray:
        '''
        :param normalize_expressions: if keras should normalize gene expressions
        :return: X array to be used as input data by keras
        '''
        data = self.samples.join(self.expressions, on = "Name", how="inner")
        ji = data.columns.drop(self.drop_list)
        x = data[ji]
        
        # adding one-hot-encoded tissues and sex
        #x = pd.concat([x,pd.get_dummies(data['Tissue'], prefix='tissue'), pd.get_dummies(data['Sex'], prefix='sex')],axis=1)
        
        steps = [('standardization', StandardScaler()), ('normalization', MinMaxScaler())]
        pre_processing_pipeline = Pipeline(steps)
        transformed_data = pre_processing_pipeline.fit_transform(x)

        x = transformed_data
        
        print('Data length', len(x))
        
        return x #normalize(x, axis=0) if normalize_expressions else x
    
    def get_features_dataframe(self, add_tissues=False):
        data = self.samples.join(self.expressions, on = "Name", how="inner")
        ji = data.columns.drop(self.drop_list)
        df = data[ji]
        if add_tissues:
            df = pd.concat([df,pd.get_dummies(data['Tissue'], prefix='tissue'), pd.get_dummies(data['Sex'], prefix='sex')],axis=1)
        x = df.values
        
        min_max_scaler = MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        df_normalized = pd.DataFrame(x_scaled, columns=df.columns, index=df.index)
        return df_normalized


### Loading data

In [3]:
samples_path = '../data/gtex/v8_samples.parquet'
expressions_path = '../data/gtex/v8_expressions.parquet'

In [4]:
genes = Genes(samples_path, expressions_path, problem_type="regression")
genes.samples.head(10)

,Tissue,Subtissue,Age,Sex,Death,Avg_age
Name,,,,,,
GTEX-111CU-0126-SM-5GZWZ,Adrenal Gland,Adrenal Gland,50-59,male,ventilator case,54.5
GTEX-111CU-0226-SM-5GZXC,Thyroid,Thyroid,50-59,male,ventilator case,54.5
GTEX-111CU-0326-SM-5GZXO,Lung,Lung,50-59,male,ventilator case,54.5
GTEX-111CU-0426-SM-5GZY1,Spleen,Spleen,50-59,male,ventilator case,54.5
GTEX-111CU-0526-SM-5EGHK,Pancreas,Pancreas,50-59,male,ventilator case,54.5
GTEX-111CU-0626-SM-5EGHL,Esophagus,Esophagus - Muscularis,50-59,male,ventilator case,54.5
GTEX-111CU-0726-SM-5GZYD,Esophagus,Esophagus - Mucosa,50-59,male,ventilator case,54.5
GTEX-111CU-0826-SM-5EGIJ,Esophagus,Esophagus - Gastroesophageal Junction,50-59,male,ventilator case,54.5
GTEX-111CU-0926-SM-5EGIK,Stomach,Stomach,50-59,male,ventilator case,54.5


In [5]:
from keras import backend as K

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def optimized_age_model_regression(x_train, x_val, y_train, y_val):
    # optimized_age_model(x_train, x_val, y_train, y_val, params: dict):
    input_layer = Input(shape=(x_train.shape[1],))
    reg = keras.regularizers.l1_l2(l1=0.3, l2=0.3)
    mod = Dense(1024, activation=relu)(input_layer) # 196
    mod = Dropout(0.1)(mod) 
    mod = Dense(512, activation=relu)(mod) # 196
    mod = Dropout(0.1)(mod)    
    mod = Dense(64, activation=relu)(mod) #64
    mod = Dropout(0.1)(mod)
    
    outputs = [Dense(1, name='age_output')(mod)] #let's try to make it simple and start with age 
    #outputs = [Dense(y_train.shape[1], activation='sigmoid', name='age_output')(mod)] #let's try to make it simple and start with age 
    loss = {'age_output': 'mse'}
    weights={'age_output': 1.0}
    metrics = {'age_output': ['mae', coeff_determination]}
    
    model = Model(inputs=input_layer, outputs=outputs)
    model.summary()
    model.compile(optimizer='adam',
              loss=loss,
              loss_weights=weights,
              metrics=metrics,
                 )

    return model


def optimized_age_model_classification(x_train, x_val, y_train, y_val):
    # optimized_age_model(x_train, x_val, y_train, y_val, params: dict):
    input_layer = Input(shape=(x_train.shape[1],))
    reg = keras.regularizers.l1_l2(l1=0, l2=0)
    mod = Dense(512, activation=relu, kernel_regularizer=reg, bias_regularizer=reg)(input_layer) # 196
    mod = Dropout(0.2)(mod)    
    mod = Dense(64, activation=relu, kernel_regularizer=reg, bias_regularizer=reg)(mod) #64
    mod = Dropout(0.2)(mod)
    
    #outputs = [Dense(1, name='age_output')(mod)] #let's try to make it simple and start with age 
    outputs = [Dense(y_train.shape[1], activation='sigmoid', name='age_output')(mod)] #let's try to make it simple and start with age 
    loss = {'age_output': 'categorical_crossentropy'}
    weights={'age_output': 1.0}
    metrics = {'age_output': 'mae'}
    
    model = Model(inputs=input_layer, outputs=outputs)
    model.summary()
    model.compile(optimizer='adam',
              loss=loss,
              loss_weights=weights,
              metrics=metrics,
                 )

    return model

Using TensorFlow backend.


In [6]:
def Huber(yHat, y, delta=1.):
    return np.where(np.abs(y-yHat) < delta,.5*(y-yHat)**2 , delta*(np.abs(y-yHat)-0.5*delta))

def transform_to_probas(age_intervals):
    class_names = ['20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
    res = []
    for a in age_intervals:
        non_zero_index = class_names.index(a)
        res.append([0 if i != non_zero_index else 1 for i in range(len(class_names))])
    return np.array(res)
    
def transform_to_interval(age_probas):
    class_names = ['20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
    return np.array(list(map(lambda p: class_names[np.argmax(p)], age_probas)))        

### Regression model CV

In [ ]:
genes = Genes(samples_path, expressions_path, problem_type="regression")
X = genes.prepare_data(True)
Y = genes.Y
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

rmse = []
mae = []
r2 = []
huber_loss = []

for train, test in kfold.split(X, Y):
    model = optimized_age_model_regression(X[train], X[test], Y[train], Y[test])
    model.fit(X[train], Y[train], validation_data=[X[test], Y[test]], epochs=150, batch_size=256, verbose=1)
    predictions = model.predict(X[test])
    test_y = Y[test]

    print("R^2", r2_score(test_y, predictions))
    print("Mean squared error", mean_squared_error(test_y, predictions))
    print("Mean absolute error", mean_absolute_error(test_y, predictions))
    print('Huber loss', np.mean(Huber(test_y, predictions)))

    rmse.append(mean_squared_error(test_y, predictions))
    mae.append(mean_absolute_error(test_y, predictions))
    r2.append(r2_score(test_y, predictions))
    huber_loss.append(np.mean(Huber(test_y, predictions)))
      
print('Mean MSE = ', np.mean(rmse))
print('Mean MAE = ', np.mean(mae))
print('Mean R2 = ', np.mean(r2))
print('Mean Huber Loss = ', np.mean(huber_loss))

Data length 15343
Model: "model_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, 18388)]           0         
_________________________________________________________________
dense_89 (Dense)             (None, 1024)              18830336  
_________________________________________________________________
dropout_89 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_90 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_90 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_91 (Dense)             (None, 64)                32832     
_________________________________________________________________
dropout_91 (Dropout)         (None, 64) 

### Classification model CV

In [31]:
genes = Genes(samples_path, expressions_path, problem_type="classification")
X = genes.get_features_dataframe().values
Y = genes.Y
kfold = KFold(n_splits=5, shuffle=True, random_state=87)

accuracies = []
f1s = []
precisions = []
recalls = []

for train, test in kfold.split(X, transform_to_interval(Y)):
    model = optimized_age_model_classification(X[train], X[test], Y[train], Y[test])
    model.fit(X[train], Y[train], validation_data=[X[test], Y[test]], epochs=5, batch_size=256, verbose=0, class_weight=class_weight.compute_class_weight('balanced',
                                             np.unique(transform_to_interval(Y)),
                                             transform_to_interval(Y)))
    predictions = transform_to_interval(model.predict(X[test]))
    test_y = transform_to_interval(Y[test])

    print("Accuracy", accuracy_score(test_y, predictions))
    print("F1 Score", f1_score(test_y, predictions, average='weighted'))
    print("Precision", precision_score(test_y, predictions, average='weighted'))
    print('Recall', recall_score(test_y, predictions, average='weighted'))

    accuracies.append(accuracy_score(test_y, predictions))
    f1s.append(f1_score(test_y, predictions, average='weighted'))
    precisions.append(precision_score(test_y, predictions, average='weighted'))
    recalls.append(recall_score(test_y, predictions, average='weighted'))

print('Mean Accuracy = ', np.mean(accuracies))
print('Mean F1 Score = ', np.mean(f1s))
print('Mean Precision = ', np.mean(precisions))
print('Mean Recall = ', np.mean(recalls))

['20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 18420)]           0         
_________________________________________________________________
dense_28 (Dense)             (None, 512)               9431552   
_________________________________________________________________
dropout_28 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 64)                32832     
_________________________________________________________________
dropout_29 (Dropout)         (None, 64)                0         
_________________________________________________________________
age_output (Dense)           (None, 6)                 390       
Total params: 9,464,774
Trainable params: 9,464,774
Non-trainable par

/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 18420)]           0         
_________________________________________________________________
dense_30 (Dense)             (None, 512)               9431552   
_________________________________________________________________
dropout_30 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 64)                32832     
_________________________________________________________________
dropout_31 (Dropout)         (None, 64)                0         
_________________________________________________________________
age_output (Dense)           (None, 6)                 390       
Total params: 9,464,774
Trainable params: 9,464,774
Non-trainable params: 0
________________________________________________